# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import sys
import time
import multiprocessing
import pprint 

from datetime import datetime, timedelta

from Packages.NoveltyDetection.setup.noveltyDetectionConfig import CONFIG
from SAENoveltyDetectionAnalysis import SAENoveltyDetectionAnalysis
from Functions.telegrambot import Bot

num_processes = multiprocessing.cpu_count()

my_bot = Bot("lisa_thebot")

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

training_params = {
    "Technique": "StackedAutoEncoder",
    "TechniqueParameters": {
        "allow_change_weights": False #False
    },
    "DevelopmentMode": False,
    "DevelopmentEvents": 1600,
    "NoveltyDetection": True,
    "InputDataConfig": {
        "database": "4classes",
        "n_pts_fft": 1024,
        "decimation_rate": 3,
        "spectrum_bins_left": 400,
        "n_windows": 1,
        "balance_data": True
    },
    "OptmizerAlgorithm": {
        "name": "Adam",
        "parameters": {
            "learning_rate": 0.001,
            "beta_1": 0.90,
            "beta_2": 0.999,
            "epsilon": 1e-08,
            "learning_decay": 1e-6,
            "momentum": 0.3,
            "nesterov": True
        }
    },
    "HyperParameters": {
        "n_folds": 4,
        "pretraining_n_epochs": 300,
        "finetuning_n_epochs": 300,
        "n_inits": 2,
        "batch_size": 64,
        "kernel_initializer": "uniform",
        "bias_initializer": "ones",
        "encoder_activation_function": "sigmoid",
        "decoder_activation_function": "sigmoid",
        "classifier_output_activation_function": "softmax",
        "norm": "mapstd",
        "metrics": ["accuracy"],
        "loss": "mean_squared_error",
        "classifier_loss": "mean_squared_error",
        "dropout": False,
        "dropout_parameter": 0.00,
        "regularization": None, 
        "regularization_parameter": 0.00
    },
    "callbacks": {
        "EarlyStopping": {
            "patience": 300,
            "monitor": "val_loss"
        }
    }
}
analysis = SAENoveltyDetectionAnalysis(parameters=training_params,
                                       model_hash="",
                                       load_hash=False, load_data=True, verbose=True)
all_data, all_trgt, trgt_sparse = analysis.getData()

SAE = analysis.createSAEModels()

trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

Using TensorFlow backend.
/home/vinicius.mello/anaconda3/envs/sonarenv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Creating /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f
Creating /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f/AnalysisFiles
Creating /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f/Pictures
Creating /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f/Models
Creating /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f/Logs
Saving /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAu

In [2]:
pp = pprint.PrettyPrinter(indent=1)
print(analysis.model_hash)
print(analysis.getBaseResultsPath())
pp.pprint(analysis.parameters)

c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f
/home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/c579f5a2dff7417197b0e2186e7b7b25e6b52d98f9e2e0e232c6ba1f56c6d68f
{'DevelopmentEvents': 1600,
 'DevelopmentMode': False,
 'HyperParameters': {'batch_size': 64,
                     'bias_initializer': 'ones',
                     'classifier_loss': 'mean_squared_error',
                     'classifier_output_activation_function': 'softmax',
                     'decoder_activation_function': 'sigmoid',
                     'dropout': False,
                     'dropout_parameter': 0.0,
                     'encoder_activation_function': 'sigmoid',
                     'finetuning_n_epochs': 300,
                     'kernel_initializer': 'uniform',
                     'loss': 'mean_squared_error',
                     'metrics': ['accuracy'],
                     'n_folds': 4,
                     'n_inits': 2,
         

### Perform the training of the model

In [ ]:
for inovelty in range(len(analysis.class_labels)):
    startTime = time.time()
    
    data=trn_data[inovelty]
    trgt=trn_trgt[inovelty]
    trainingType = "neuronSweep" #foldSweep, neuronSweep, normal, layerSweep
    analysis.train(layer=1,
                   inovelty=inovelty,
                   fineTuning=True,
                   trainingType=trainingType,
                   hidden_neurons=[400],
                   neurons_variation_step=50,
                   numThreads=8,
                   model_hash=analysis.model_hash)
    
    duration = str(timedelta(seconds=float(time.time() - startTime)))
    print("The training of the model for novelty class {0} took {1} to be performed\n".format(analysis.class_labels[inovelty], duration))

python /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Packages/NoveltyDetection/StackedAutoEncoders/sae_train.py --layer 1 --novelty 0 --finetunning 1 --threads 8 --type neuronSweep --hiddenNeurons 400 --neuronsVariationStep 50 --modelhash d7e9c272a78cd00c3e4ad161fa3cae389c3b2547aff986dd65669d5bceb5f421
The training of the model for novelty class A took 6:39:51.959771 to be performed

python /home/vinicius.mello/Workspace/LPS/SonarAnalysis/Packages/NoveltyDetection/StackedAutoEncoders/sae_train.py --layer 1 --novelty 1 --finetunning 1 --threads 8 --type neuronSweep --hiddenNeurons 400 --neuronsVariationStep 50 --modelhash d7e9c272a78cd00c3e4ad161fa3cae389c3b2547aff986dd65669d5bceb5f421


In [156]:
analysis.get_analysis_zip()

'/home/vinicius.mello/Workspace/LPS/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/outputs/f84a0f006d8a19e5c3dbe25d4907659013495f79bb8b377e88673a899d9e2a2d/AnalysisFiles.zip'